In [56]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
from math import isnan
import matplotlib.pyplot as plt
import random
%matplotlib inline

try:
    xrange = xrange
except:
    xrange = range

In [74]:
class enviroment():
    def __init__(self,size,goal):
        self.size_x=size[0]
        self.size_y=size[1]
        self.goal_x=goal[0]
        self.goal_y=goal[1]
    def reset(self):
        self.pos_x=random.randint(1, self.size_x)
        self.pos_y=random.randint(1, self.size_y)
        self.start_x=self.pos_x
        self.start_y=self.pos_y
        return np.array([self.pos_x,self.pos_y])
    
    def at_goal(self,change_x,change_y):
        if(self.pos_x+change_x==self.goal_x and self.pos_y+change_y==self.goal_y):
            return True
        else:
            return False
    
    def wrong_move(self,c_x,c_y):
        if(self.pos_x+c_x<1 or self.pos_x+c_x>self.size_x or self.pos_y+c_y<1 or self.pos_y+c_y>self.size_y):
            return True
        else:
            return False
    
    def positive(self,c_x,c_y):
        if(abs(self.goal_x-self.pos_x)+abs(self.goal_y-self.pos_y)-(abs(self.goal_x-(self.pos_x+c_x))+abs(self.goal_y-(self.pos_y+c_y)))>0):
            return True
        else:
            return False
        
    def step(self,a):
        if(a==0):
            c_x=0
            c_y=-1
        elif(a==1):
            c_x=-1
            c_y=0
        elif(a==2):
            c_x=0
            c_y=1
        elif(a==3):
            c_x=1
            c_y=0
        else:
            print("Kaboom")
            
        if self.at_goal(c_x,c_y):
            return (np.array([self.pos_x+c_x, self.pos_y+c_y]),+10000,True)

        if self.wrong_move(c_x,c_y):
            self.pos_x=self.pos_x
            self.pos_y=self.pos_y
            return np.array([self.pos_x,self.pos_y]),-100,False
        elif self.positive(c_x,c_y):
            self.pos_x=self.pos_x+c_x
            self.pos_y=self.pos_y+c_y
            return np.array([self.pos_x,self.pos_y]),+1,False
        else:
            self.pos_x=self.pos_x+c_x
            self.pos_y=self.pos_y+c_y
            return np.array([self.pos_x,self.pos_y]),-1,False
                                

In [75]:
gamma = 0.99

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r) #DICOUNT_SIZE=R_SIZE
    running_add = 0 #INITIALIZE
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t] #new_reward=reward+gamma*previous_reward
        discounted_r[t] = running_add
    return discounted_r

In [76]:
class agent():
    def __init__(self, lr, s_size,a_size,h_size): #_,State_size,action_size,hidden layer_size
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        self.state_in= tf.placeholder(shape=[None,s_size],dtype=tf.float64) # Placeholder to hold a float32 vector of size sstate size
        hidden = slim.fully_connected(self.state_in,h_size,biases_initializer=None,activation_fn=tf.nn.relu)
        self.output = slim.fully_connected(hidden,a_size,activation_fn=tf.nn.softmax,biases_initializer=None)
        self.chosen_action = tf.argmax(self.output,1)
        
        #NN is [s_size->(nn.relu) hidden_layer_size->(nn.softmax) action_size-> max_probablity]
        
        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
        #Placeholders for reward and action
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float64)
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        
        #Creates an increasing array of [0,1,.. size of(output) in 0th axis] * size  of (output) in 1st axis + action_holder
        self.indexes = (tf.range(0, tf.shape(self.output)[0]) * (tf.shape(self.output)[1])) + self.action_holder 
        #Gathers the important outputs, reshapes output to 1 hot vector, and select important elements by by self.indexes
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)  

        # Loss = mean(log(outputs)*rewards) #Reduce mean computes mean of given set of vecotrs
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs)*self.reward_holder)
        
        tvars = tf.trainable_variables()
        #Gradient Holder
        self.gradient_holders = []
        #http://book.pythontips.com/en/latest/enumerate.html
        for idx,var in enumerate(tvars):
            #Placeholder for every trainable variable
            placeholder = tf.placeholder(tf.float64,name=str(idx)+'_holder')
            self.gradient_holders.append(placeholder)
        #Computes gradient of loss wrt tvars
        self.gradients = tf.gradients(self.loss,tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        #https://www.programiz.com/python-programming/methods/built-in/zip
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,tvars))

In [80]:
tf.reset_default_graph() #Clear the Tensorflow graph.

env=enviroment(np.array([5,5]),np.array([3,3]))
a_siz=4
myAgent = agent(lr=1e-2,s_size=2,a_size=a_siz,h_size=16) #Load the agent.

total_episodes = 10000 #Set total number of episodes to train agent on.
max_ep = 300 
update_frequency = 1
# init_epsilon = 0.95
epsilon = 0.85

init = tf.global_variables_initializer()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init) #start
    i = 0
    total_reward = []
    total_length = []
        
    gradBuffer = sess.run(tf.trainable_variables())

    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0 #Gradient initialize to zero.

    while i < total_episodes:
        s = env.reset() #Reset the enviroment ange get the states
        epsilon=init_epsilon+i*((1-init_epsilon)/5000)
        running_reward = 0
        ep_history = []
#         print("Episode:{}".format(i))
        for j in range(max_ep):
            #Probabilistically pick an action given our network outputs.
            a_dist = sess.run(myAgent.output,feed_dict={myAgent.state_in:[s]}) #Run the neural net with input state as s
#             if(j==0):
#             print("a_dist:{} s:{}".format(a_dist,s))
            
            a = np.random.choice(a_dist[0],p=a_dist[0])# Make a Random choice between element of a_dist with probabilities of that element
            a = np.argmax(a_dist == a)#Check the index of the random choice
            
            if(np.random.rand(1)>epsilon):
                a=np.random.randint(a_siz, size=1)
            
            
            s1,r,done = env.step(a) #Get our reward for taking an action given a bandit.
            ep_history.append([s,a,r,s1]) #Append the episode history
            s = s1
            running_reward += r #r is reward
#             print("j:{} a:{} s:{} r:{}".format(j,a,s1,r))
            if done == True:
                #Update the network.
                #Convert into numpy array
                ep_history = np.array(ep_history)
                #Calculate discount rewards based on walue of r
                ep_history[:,2] = discount_rewards(ep_history[:,2])
#                 print("Disount_rewards:{}".format(ep_history[:,2]))
                #Feed Dictionary consisting of episode history[reward] -> reward holder, episode history[action] -> actionholder and episode history[state]->state
                feed_dict={myAgent.reward_holder:ep_history[:,2],
                        myAgent.action_holder:ep_history[:,1],myAgent.state_in:np.vstack(ep_history[:,0])}
                grads = sess.run(myAgent.gradients, feed_dict=feed_dict) #Compute gradient
                for idx,grad in enumerate(grads):
                    gradBuffer[idx] += grad

                if i % update_frequency == 0 and i != 0:
                    feed_dict= dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                    _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                    for ix,grad in enumerate(gradBuffer):
                        gradBuffer[ix] = grad * 0
                
                total_reward.append(running_reward)
                total_length.append(j)
                break

        
            #Update our running tally of scores.
        if i % 100 == 0:
            print("episode:{} start:{} j:{} a:{} s:{} r:{}".format(i,np.array([env.start_x,env.start_y]), j,a,s1,np.mean(total_reward[-100:])))
        i+= 1
        
    
#     Testing
    s = env.reset() #Reset the enviroment ange get the states
    print("init_state:{}".format(s))
    running_reward = 0
    
    for j in range(max_ep):
        #Probabilistically pick an action given our network outputs.
        a_dist = sess.run(myAgent.output,feed_dict={myAgent.state_in:[s]}) #Run the neural net with input state as s
        a = np.random.choice(a_dist[0],p=a_dist[0])# Make a Random choice between element of a_dist with probabilities of that element
        a = np.argmax(a_dist == a)#Check the index of the random choice

        s1,r,done = env.step(a) #Get our reward for taking an action given a bandit.
        s = s1
        running_reward += r #r is reward
        print("j:{} a:{} s:{} r:{}".format(j,a,s1,r))
        if done == True:
            print("Solved")
            break

episode:0 start:[1 4] j:2 a:3 s:[3 3] r:10002.0
episode:100 start:[3 1] j:42 a:3 s:[3 3] r:8528.681818181818
episode:200 start:[4 3] j:0 a:1 s:[3 3] r:9672.4
episode:300 start:[4 1] j:72 a:2 s:[3 3] r:9280.4
episode:400 start:[3 5] j:7 a:1 s:[3 3] r:9619.26
episode:500 start:[3 4] j:30 a:1 s:[3 3] r:9740.34
episode:600 start:[1 4] j:2 a:0 s:[3 3] r:9774.44
episode:700 start:[5 4] j:11 a:0 s:[3 3] r:9814.58
episode:800 start:[5 1] j:3 a:1 s:[3 3] r:9398.43
episode:900 start:[1 2] j:11 a:3 s:[3 3] r:9762.5
episode:1000 start:[3 1] j:14 a:2 s:[3 3] r:9896.55
episode:1100 start:[2 1] j:6 a:3 s:[3 3] r:8804.25
episode:1200 start:[2 2] j:6 a:3 s:[3 3] r:9120.29
episode:1300 start:[2 3] j:0 a:3 s:[3 3] r:8593.23
episode:1400 start:[4 2] j:28 a:0 s:[3 3] r:9628.43
episode:1500 start:[3 2] j:0 a:2 s:[3 3] r:9837.42
episode:1600 start:[3 5] j:3 a:1 s:[3 3] r:9791.5
episode:1700 start:[4 1] j:4 a:0 s:[3 3] r:9830.31
episode:1800 start:[1 3] j:19 a:0 s:[3 3] r:9831.29
episode:1900 start:[4 5] j:2 